<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!apt install 7zip
!python -m pip install --upgrade pip setuptools wheel
!pip install libtorrent lbry-libtorrent dropbox tomlkit
# !python -m pip install lbry-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
7zip is already the newest version (21.07+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Imports
rich.progress for concurrent progress tracking<br>
libtorrent as torrent client<br>
dropbox as storage solution<br>

In [100]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed

import time
from timeit import default_timer as timer

from pathlib import Path
from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

import libtorrent as lt
from dropbox import Dropbox, DropboxOAuth2FlowNoRedirect

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn

# Setup Filesystem

In [ ]:
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())
# dbxcfg_path = Path("/content/dbx.ini")
# dbxcfg_file = TOMLFile(dbxcfg_file.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("torrents stored in a folder or are a single file"))
  magnet_document.add("show1fldr", example_folder_tor)
  magnet_document.add("movie1", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("show2fldr", example_multifiles)

  magnet_file.write(magnet_document)

# dbx_conf = ConfigParser()
# if not dbxcfg_path.exists():
#   dbx_conf["AUTHENTICATION"] = {
#       "APP_KEY":"",
#       "APP_SECRET":""
#   }
#   with dbxcfg_path.open("w") as fw:
#     dbx_conf.write(fw)

# Download Torrents

In [ ]:
# TODO: Handle torrent files
def download_torrent(progress: Progress, media_name: str, link: str):
  MAX_FILENAME_LEN = 35
  METADATA_TIMEOUT_MS = .8
  save_path=Path("/content/Torrents/") / media_name
  if not save_path.exists():
    save_path.mkdir(parents=True)

  task = progress.add_task(
        "download",
        total=100,
        status="[yellow]Obtaining Metadata",
        filename=media_name,
        speed="0.0 kB/s",
  )
  ses = lt.session()

  # setup torrent handle
  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    progress.update(task, status="[red]Failed to parse magnet!")
    return None
  atp.save_path = str(save_path.as_posix())

  handle = ses.add_torrent(atp)
  status = handle.status()

  while not handle.status().has_metadata:
    time.sleep(METADATA_TIMEOUT_MS/1000)
    status = handle.status()

  filename =  status.name if len(status.name) < MAX_FILENAME_LEN \
              else f"{status.name[:MAX_FILENAME_LEN]}..."

  progress.update(task,
                  status="[yellow]Downloading",
                  filename=filename
  )

  # download torrent
  while not status.is_seeding:
    status = handle.status()

    progress.update(task,
                completed=status.progress * 100,
                speed=f"{status.download_rate/1000:.1f} kB/s",
    )
  progress.update(task,
                  status="[green]Complete",
                  completed=100,
  )

  return save_path

In [117]:
def zipfiles(progress, m, magnet_links):
  with ThreadPoolExecutor() as executor:
    futures = [executor.submit(download_torrent, progress, m, link) for link in magnet_links]
    save_path = Path("/content/Torrents/") / m

    for future in as_completed(futures):
      try:
        future.result()
      except Exception as e:
        print("Upload failed:", e)
        return

  subprocess.run(["7za", "a", "-v1g", f"/content/transfer/{save_path.name}.zip", save_path.as_posix()])

with Progress(
  TextColumn("{task.fields[status]}"),
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
) as progress:

  with ThreadPoolExecutor() as executor:
    media = magnet_file.read()
    futures = []
    for m in media:
      magnet_links = media[m]["magnets"]

      futures.append(executor.submit(zipfiles, progress, m, magnet_links))

    for future in as_completed(futures):
      try:
          future.result()
      except Exception as e:
          print("Upload failed:", e)

Output()

In [ ]:
# def upload(
#     access_token,
#     file_path,
#     target_path,
#     timeout=900,
#     chunk_size=4 * 1024 * 1024,
# ):
#     dbx = dropbox.Dropbox(access_token, timeout=timeout)
#     with open(file_path, "rb") as f:
#         file_size = os.path.getsize(file_path)
#         if file_size <= chunk_size:
#             print(dbx.files_upload(f.read(), target_path))
#         else:
#             with tqdm(total=file_size, desc="Uploaded") as pbar:
#                 upload_session_start_result = dbx.files_upload_session_start(
#                     f.read(chunk_size)
#                 )
#                 pbar.update(chunk_size)
#                 cursor = dropbox.files.UploadSessionCursor(
#                     session_id=upload_session_start_result.session_id,
#                     offset=f.tell(),
#                 )
#                 commit = dropbox.files.CommitInfo(path=target_path)
#                 while f.tell() < file_size:
#                     if (file_size - f.tell()) <= chunk_size:
#                         print(
#                             dbx.files_upload_session_finish(
#                                 f.read(chunk_size), cursor, commit
#                             )
#                         )
#                     else:
#                         dbx.files_upload_session_append(
#                             f.read(chunk_size),
#                             cursor.session_id,
#                             cursor.offset,
#                         )
#                         cursor.offset = f.tell()
#                     pbar.update(chunk_size)

# torrents = list(Path("/content/").glob("Torrent.zip.*"))
# # upload(TOKEN, torrents[1].as_posix(), "/Torrent/" + torrents[1].name)
# # for torrent in torrents:


# # run uploads in parallel
# with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#     futures = [
#         executor.submit(upload, TOKEN, torrent.as_posix(), "/Torrent/" + torrent.name)
#         for torrent in torrents
#     ]
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             future.result()
#         except Exception as e:
#             print("Upload failed:", e)

Uploaded: 931135488it [03:34, 4341456.09it/s]

FileMetadata(client_modified=datetime.datetime(2025, 10, 6, 6, 35, 57), content_hash='7da9e45bf034cf26c2e008db41876fc4150e4e0eef238532d2e7ea92cc9631cd', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:uarvdFOTHrAAAAAAAAAAHg', is_downloadable=True, media_info=NOT_SET, name='Torrent.zip.001', parent_shared_folder_id=NOT_SET, path_display='/Torrent/Torrent.zip.001', path_lower='/torrent/torrent.zip.001', preview_url=NOT_SET, property_groups=NOT_SET, rev='0164077aa05e6c60000000300d37ac1', server_modified=datetime.datetime(2025, 10, 6, 6, 35, 58), sharing_info=NOT_SET, size=928419183, symlink_info=NOT_SET)
